C# has two separate mechanisms for writing code that is `reusable` across different types: `inheritance` and `generics`.

generics express reusability with a `template` that contains `placeholder` types.

### Generic Types

A generic type declares type `parameters—placeholder` types to be `filled` in by the `consumer` of the generic type, which supplies the `type arguments`.

In [ ]:
public class Stack<T>
{
    int position;
    T[] data = new T[100];
    public void Push (T obj) => data[position++] = obj;
    public T Pop() => data[--position];
}

var stack = new Stack<int>();
stack.Push (5);
stack.Push (10);
int x = stack.Pop(); // x is 10
int y = stack.Pop(); // y is 5

Technically, we say that `Stack<T>` is an `open type`, whereas `Stack<int>` is a `closed type`. ***At runtime***, `all generic type instances are closed`

### Why Generics Exist

Generics exist to `write code` that is `reusable` across `different types`.

In [ ]:
//without generic , we must create stack for each type or:
public class ObjectStack
{
    int position;
    object[] data = new object[10];
    public void Push (object obj) => data[position++] = obj;
    public object Pop() => data[--position];
}

// Suppose we just want to store integers here:
ObjectStack stack = new ObjectStack();
stack.Push ("s"); // Wrong type, but no error!
int i = (int)stack.Pop(); // Downcast - runtime error

with generics we use stack to a `specific element type` for increased `type safety` and `reduced` `casting` and `boxing`.

### Generic Methods

In [ ]:
static void Swap<T> (ref T a, ref T b)
{
    T temp = a;
    a = b;
    b = temp;
}

int x = 5;
int y = 10;
Swap (ref x, ref y); //no need to supply type arguments 
//compiler can implicitly infer the type.

`Methods` and `types` are the `only` constructs that can introduce type parameters. Properties, indexers, events, fields, constructors, operators, and so on `cannot` declare type parameters

they can use generic of `enclosing type`.

### Declaring Type Parameters

Type parameters can be introduced in the declaration of `classes`, `structs`, `interfaces`, `delegates` , and `methods`.

In [ ]:
public struct Nullable<T>
{
    public T Value { get; } // property can use type parameter
}

//A generic type or method can have multiple parameters:
class Dictionary<TKey, TValue> {}

//To instantiate:
Dictionary<int,string> myDict = new Dictionary<int,string>();


//Generic type names and method names can be overloaded as long as the number
//of type parameters is different.
class A {}
class A<T> {}
class A<T1,T2> {}

//By convention, generic types and methods with a single type
//parameter typically name their parameter T
/*When using multiple type parameters, 
each parameter is prefixed with T but has a more descriptive name.*/

### typeof and Unbound Generic Types

`Open generic` types do not exist at `runtime`: they are `closed` as part of `compilation`.

it is `possible` for an `unbound generic` type to exist at `runtime`—`purely as a Type object`.

In [1]:
class A<T> {}
class A<T1,T2> {}

Type a1 = typeof (A<>); // Unbound type (notice no type arguments).
Type a2 = typeof (A<,>); // Use commas to indicate multiple type args.

Console.WriteLine(a1.ToString());
Console.WriteLine(a2.ToString());

//You can also use the typeof operator to specify a closed type:
Type a3 = typeof (A<int,int>);

//very useful for DI

//Or, you can specify an open type (which is closed at runtime):
class B<T> { void X() { Type t = typeof (T); } }

Submission#2+A`1[T]
Submission#2+A`2[T1,T2]


### The default Generic Value

In [2]:
//The default value for a reference type is null
//default value for a value type is the result of 
//bitwise-zeroing the value type’s fields

static void Zap<T> (T[] array)
{
    for (int i = 0; i < array.Length; i++)
        array[i] = default(T);
    /*From C# 7.1, you can omit the type argument for cases in which 
    the compiler is ableto infer it. 
    We could replace the last line of code with this:
        array[i] = default;*/
}

<div dir="rtl" style="width: 90%;">

عبارت "bitwise-zeroing" به فرآیندی اشاره دارد که در آن تمام بیت‌های حافظه‌ای که به نوع مقداری اختصاص یافته، به صفر تنظیم می‌شوند. این بدین معناست که هر فیلد درون ساختار یا نوع مقداری به مقدار صفر اولیه‌سازی می‌شود. برای انواع ابتدایی مانند int یا double، این به معنای صفر عددی است. برای انواع بولین (bool), صفر به معنای false است.
</div>

### Generic Constraints

In [ ]:
public class GenericRepository<TEntity>
    where TEntity : Entity
{}

where T : base-class // Base-class constraint
where T : interface // Interface constraint
where T : class // Reference-type constraint
where T : class? // (see "Nullable Reference Types" in Chapter 1)
where T : struct // Value-type constraint (excludes Nullable types)
where T : unmanaged // Unmanaged constraint
where T : new() // Parameterless constructor constraint
where U : T // Naked type constraint

In [ ]:
public class TestConstraint<T> where T : class
{ }

public class TestConstraint1<T> where T : TestConstraint<T>
{ }

public class TestConstraint2<T> where T : IEnumerable<T>
{ }

public class TestConstraint3<T> where T : class?
{ }

public class TestConstraint4<T> where T : struct
{ }

public class TestConstraint5<T> where T : unmanaged // T is not refrence type and if is value type, dont refrence to referrenc type
{ }

public class TestConstraint6<T> where T : new() // jsut types can instanciated
{ }

public class TestConstraint7<T, U> where U : T
{ }

public class TestConstraint7<T> where T : notnull
{ }

In [ ]:
class GenericClass<T,U> where T : SomeClass, Interface1
where U : new()

static T Max <T> (T a, T b) where T : IComparable<T>
{
    return a.CompareTo (b) > 0 ? a : b;
}

### Subclassing Generic Types

In [ ]:
class Stack<T> {}
class SpecialStack<T> : Stack<T> {}

//A subtype can also introduce fresh type arguments:
class List<T> {}
class KeyedList<T,TKey> : List<T> {}

### Self-Referencing Generic Declarations

In [ ]:
//A type can name itself as the concrete type when closing a type argument:
public interface IEquatable<T> { bool Equals (T obj); }
public class Balloon : IEquatable<Balloon>
{
    public string Color { get; set; }
    public int CC { get; set; }
    public bool Equals (Balloon b)
    {
        if (b == null) return false;
        return b.Color == Color && b.CC == CC;
    }
}

### Static Data

In [ ]:
//Static data is unique for each closed type:
Console.WriteLine (++Bob<int>.Count); // 1
Console.WriteLine (++Bob<int>.Count); // 2
Console.WriteLine (++Bob<string>.Count); // 1
Console.WriteLine (++Bob<object>.Count); // 1
class Bob<T> { public static int Count; }

### Type Parameters and Conversions

C#’s cast operator can perform several kinds of conversion, including:
- Numeric conversion
- Reference conversion
- Boxing/unboxing conversion
- Custom conversion